In [3]:
from pathlib import Path
patterns = Path('./patterns')
choices = list(patterns.glob("*"))
for i,pattern in enumerate(choices):
    print(i+1,pattern.name)

1 new_plugin_with_types
2 new_tool


In [4]:
choice = int(input(f"Choose a pattern ({1}-{len(choices)}): "))
assert 1 <= choice <= len(choices)
pattern = choices[choice-1]
print("You chose",pattern.name)

You chose new_plugin_with_types


In [5]:
pyfile = pattern / "__pattern__.py"
assert pyfile.exists(), f"{pyfile} does not exist"
import importlib
spec = importlib.util.spec_from_file_location("module.name", pyfile)
module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)

# Check if the function exists, call it, and store the result
assert hasattr(module, 'gather_global_variables')
global_variables: dict[str,str] = module.gather_global_variables()
print(global_variables)

{'crate_name': 'process_monitor', 'crate_name_pascal': 'ProcessMonitor'}


In [6]:
# Clean up __pycache__ directories under patterns
import shutil
for pycache in patterns.glob("**/__pycache__"):
    print(f"Removing {pycache}")
    shutil.rmtree(pycache)

Removing patterns\new_plugin_with_types\__pycache__


In [7]:
def replace_globals(text: str) -> str:
    for k,v in global_variables.items():
        text = text.replace("{{" + k + "}}",v.strip())
    return text

In [8]:
PROJECT_ROOT = Path("..")

In [9]:
matchers = Path("matchers")

In [10]:
template_files = [x for x in pattern.glob("**/*.*") if not any(y in str(x) for y in ["__pycache__", "__pattern__.py"])]
for template in template_files:
    destination = PROJECT_ROOT / Path(*[replace_globals(x) for x in template.parts[2:]])

    # start with clone of global variables
    variables = global_variables.copy()

    # if the pattern includes other variables, we need to load the matcher to get them
    matcher = matchers / template.relative_to(patterns).with_name(template.name + ".py")
    if matcher.exists():
        spec = importlib.util.spec_from_file_location("module.name", matcher)
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)
        assert hasattr(module, 'gather_variables')
        variables.update(module.gather_variables(destination.read_text()))
        

    print(f"Applying {template} to {destination} with {len(variables)} variables")
    text = template.read_text()
    for k,v in variables.items():
        text = text.replace("{{" + k + "}}",v.strip())
    destination.parent.mkdir(parents=True, exist_ok=True)
    destination.write_text(text)

Applying patterns\new_plugin_with_types\Cargo.toml to ..\Cargo.toml with 4 variables
Applying patterns\new_plugin_with_types\crates\plugins\Cargo.toml to ..\crates\plugins\Cargo.toml with 4 variables
Applying patterns\new_plugin_with_types\crates\plugins\src\lib.rs to ..\crates\plugins\src\lib.rs with 5 variables
Applying patterns\new_plugin_with_types\crates\{{crate_name}}\Cargo.toml to ..\crates\process_monitor\Cargo.toml with 2 variables
Applying patterns\new_plugin_with_types\crates\{{crate_name}}\src\lib.rs to ..\crates\process_monitor\src\lib.rs with 2 variables
Applying patterns\new_plugin_with_types\crates\{{crate_name}}\src\{{crate_name}}_plugin.rs to ..\crates\process_monitor\src\process_monitor_plugin.rs with 2 variables
Applying patterns\new_plugin_with_types\crates\{{crate_name}}_types\Cargo.toml to ..\crates\process_monitor_types\Cargo.toml with 2 variables
Applying patterns\new_plugin_with_types\crates\{{crate_name}}_types\src\lib.rs to ..\crates\process_monitor_types\sr

In [11]:
# Clean up __pycache__ directories under matchers
import shutil
for pycache in matchers.glob("**/__pycache__"):
    print(f"Removing {pycache}")
    shutil.rmtree(pycache)

Removing matchers\new_plugin_with_types\__pycache__
Removing matchers\new_plugin_with_types\crates\plugins\__pycache__
Removing matchers\new_plugin_with_types\crates\plugins\src\__pycache__
